In [ ]:
# load ContGridMod package
using ContGridMod

In [ ]:
# load the European border file
border, scale_factor = import_border("../data/borders/border.json");

In [ ]:
# create the mesh
dx = 0.005
mesh = get_mesh(border, dx);

In [ ]:
# distribute parameters
contmod = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10, dmax=0.01, patch=1900.0, bmin=52.0, tau=5.0e-7)